In [1]:
import json, os, sys
from pprint import pprint

print("cwd = %s" % os.getcwd())

cwd = /Users/yudi/Projects/json-c


In [2]:
cocci = ''.join(open(os.path.abspath('./patch.cocci')).readlines())

def modify_src(source, verbose=False):
    if verbose:
        pprint(source)
    patch = cocci
    f = open('.temp.cocci', 'w')
    f.write(''.join(patch))
    f.close()
    execute = '/usr/local/bin/spatch -sp_file .temp.cocci -in_place %s >/dev/null 2>/dev/null' % source
    if verbose:
        execute = '/usr/local/bin/spatch -sp_file .temp.cocci %s' % source
    os.system(execute)
    
modify_src(os.path.abspath('./json_tokener.c'), verbose=True)

'/Users/yudi/Projects/json-c/json_tokener.c'


init_defs_builtins: /usr/local/Cellar/coccinelle/1.1.1_1/lib/coccinelle/standard.h
HANDLING: /Users/yudi/Projects/json-c/json_tokener.c


--- /Users/yudi/Projects/json-c/json_tokener.c
+++ /tmp/cocci-output-23807-58c2c9-json_tokener.c
@@ -16,6 +16,7 @@
 #include "config.h"
 
 #include "math_compat.h"
+#include "sandbox.h"
 #include <assert.h>
 #include <errno.h>
 #include <limits.h>
@@ -153,22 +154,33 @@ struct json_tokener *json_tokener_new_ex(int depth)
 {
 	struct json_tokener *tok;
 
+	sandbox_check_access(&(tok));
 	tok = (struct json_tokener *)calloc(1, sizeof(struct json_tokener));
+	sandbox_register_var(json_tokener_new_ex, tok, tok,
+			     (1) * (sizeof(struct json_tokener)));
 	if (!tok)
 		return NULL;
+	sandbox_check_access(&(tok->stack));
 	tok->stack = (struct json_tokener_srec *)calloc(depth, sizeof(struct json_tokener_srec));
+	sandbox_register_var(json_tokener_new_ex, tok->stack, tok->stack,
+			     (depth) * (sizeof(struct json_tokener_srec)));
 	if (!tok->stack)
 	{
+		sandbox_unregister_var(tok);
 		free(tok);
 		return NULL;
 	}
+	sandbox_check_access(&(tok->pb));
 	tok->pb = printbuf_new();
 	if 

diff = 
